In [2]:
import plotly 
import plotly.plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import cufflinks as cf
print(cf.__version__)
# Configure cufflings 
cf.set_config_file(offline=False, world_readable=True, theme='pearl')

0.12.1


In [3]:
import nltk
# nltk.download()
# !pip install autocorrect
from autocorrect import spell

In [4]:
import pandas as pd
import numpy as np
import gzip

### Data Loading

In [5]:
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

In [6]:
def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

In [7]:
df = getDF('/Users/falehalrashidi/Downloads/reviews_Books_5.json.gz')

In [8]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A10000012B7CGYKOMPQ4L,000100039X,Adam,"[0, 0]",Spiritually and mentally inspiring! A book tha...,5.0,Wonderful!,1355616000,"12 16, 2012"
1,A2S166WSCFIFP5,000100039X,"adead_poet@hotmail.com ""adead_poet@hotmail.com""","[0, 2]",This is one my must have books. It is a master...,5.0,close to god,1071100800,"12 11, 2003"
2,A1BM81XB4QHOA3,000100039X,"Ahoro Blethends ""Seriously""","[0, 0]",This book provides a reflection that you can a...,5.0,Must Read for Life Afficianados,1390003200,"01 18, 2014"
3,A1MOSTXNIO5MPJ,000100039X,Alan Krug,"[0, 0]",I first read THE PROPHET in college back in th...,5.0,Timeless for every good and bad time in your l...,1317081600,"09 27, 2011"
4,A2XQ5LZHTD4AFT,000100039X,Alaturka,"[7, 9]",A timeless classic. It is a very demanding an...,5.0,A Modern Rumi,1033948800,"10 7, 2002"


In [20]:
df1 = df[['reviewerID','asin','reviewText','helpful']]

In [21]:
df1.head()

,reviewerID,asin,reviewText,helpful
0,A10000012B7CGYKOMPQ4L,000100039X,Spiritually and mentally inspiring! A book tha...,"[0, 0]"
1,A2S166WSCFIFP5,000100039X,This is one my must have books. It is a master...,"[0, 2]"
2,A1BM81XB4QHOA3,000100039X,This book provides a reflection that you can a...,"[0, 0]"
3,A1MOSTXNIO5MPJ,000100039X,I first read THE PROPHET in college back in th...,"[0, 0]"
4,A2XQ5LZHTD4AFT,000100039X,A timeless classic. It is a very demanding an...,"[7, 9]"


In [22]:
# Create new Column for the denominator
df2 = df1.assign(denom = df1['helpful'].apply(lambda enum_denom:enum_denom[1]))

In [35]:
# Focus on reviews with at least 10 ratings on helpfulness
df3 = df2.loc[df2['denom'] > 10]
len(df3)

713836

In [36]:
# Drop unnecessary columns & reset index
df4 = df3.drop(columns=['helpful','denom'])
df4 = df4.reset_index().drop(columns=['index'])
df4.head()

,reviewerID,asin,reviewText
0,A1NPNGWBVD9AK3,000100039X,This is one of the first (literary) books I re...
1,A1SP45I55GQIIE,000100039X,"Certainly the words are of Kahlil Gibran, but ..."
2,A2ZB1G1KUE6OS6,000100039X,"I evidently misread the writeup, I thought it ..."
3,ATAPOART4QGVO,000100039X,Maybe I just wasn't in the right mood for a he...
4,A2052JNVUPRTMT,000100039X,Gibran gets right down to the bedrock of what ...


### Data Normalisation - Phase 1
* Tokenize all reviews & convert all tokens to lowecase.
* Eliminate punctuation. 
* Changing numbers into words. 
* Expanding abbreviations. 
* Canonicalisation of text. 

In [ ]:
df4.loc[df4['reviewText']] = df4['reviewText'].apply(lambda review: nltk.word_tokenize(review))

In [ ]:
import re
import string

def eliminate_punctuation(review, regex):
    new_review = []
    for token in review:
        new_token = regex.sub(u'', token)
        if not new_token == u'':
            new_review.append(new_token)
    return new_review

In [ ]:
regex=re.compile('[%s]' % re.escape(string.punctuation))
df4.loc[df1['reviewText']] = df4['reviewText'].apply(lambda review: eliminate_punctuation(review, regex))